https://enrico-alemani.medium.com/flatten-nested-dictionaries-in-pandas-using-glom-7948345c88f5
https://stackoverflow.com/questions/40588852/pandas-read-nested-json
https://stackoverflow.com/questions/35932060/dataframe-of-dataframes-in-python-pandas
https://www.statology.org/pandas-keep-columns/
https://chriswilcox.dev/blog/2020/09/02/Diving_into_GitHub_with_BigQuery_and_Python.html


In [1]:
import pandas as pd
from glom import glom
import json
import numpy as np
import gzip
from pandas import json_normalize

In [8]:
import logging

In [25]:
f = open('raw_2022-03-25-0.json')

In [26]:
df = pd.read_json(f, lines=True)

In [27]:
df1 = pd.DataFrame(df.iloc[0])

In [28]:
df1

,0
id,20926542665
type,WatchEvent
actor,"{'id': 15946300, 'login': 'TakuyaSama', 'displ..."
repo,"{'id': 158927812, 'name': 'Peltoche/lsd', 'url..."
payload,{'action': 'started'}
public,True
created_at,2022-03-25 00:00:00+00:00
org,NaN


In [29]:
df.isna().any()

id            False
type          False
actor         False
repo          False
payload       False
public        False
created_at    False
org            True
dtype: bool

In [30]:
df.isnull().any()

id            False
type          False
actor         False
repo          False
payload       False
public        False
created_at    False
org            True
dtype: bool

In [2]:
schema = [
    {
        "name": "id",
        "type": "STRING",
        "description": "Unique event ID"
    },
    {
        "name": "type",
        "type": "STRING",
        "description": "https://developer.github.com/v3/activity/events/types/"
    },
    {
        "name":"actor_id",
        "type": "INTEGER",
        "description": "Numeric ID of the GitHub actor"
    },
    {
        "name": "actor_login",
        "type": "STRING",
        "description": "Actor's GitHub login"
    },
    {
        "name": "actor_gravatar_id",
        "type": "STRING",
        "description": "Actor's Gravatar ID"
    },
    {
        "name": "actor_url",
        "type": "STRING",
        "description": "Actor's profile URL"
    },
    {
        "name": "actor_avatar_url",
        "type": "STRING",
        "description": "Actor's Gravatar URL"
    },
    {
        "name": "repo_id",
        "type": "INTEGER",
        "description": "Numeric ID of the GitHub repository"
    },
    {
        "name": "repo_name",
        "type": "STRING",
        "description": "Repository name"
    },
    {
        "name": "repo_url",
        "type": "STRING",
        "description": "Repository URL"
    },
    {
        "name": "public",
        "type": "BOOLEAN",
        "description": "Always true for this dataset since only public activity is recorded."
    },
    {
        "name": "created_at",
        "type": "TIMESTAMP",
        "description": "Timestamp of associated event"
    },
    {
        "name": "org_id",
        "type": "INTEGER",
        "description": "Numeric ID of the GitHub org"
    },
    {
        "name": "org_login",
        "type": "STRING",
        "description": "Org's GitHub login"
    },
    {
        "name": "org_gravatar_id",
        "type": "STRING",
        "description": "Org's Gravatar ID"
    },
    {
        "name": "org_avatar_url",
        "type": "STRING",
        "description": "Org's Gravatar URL"
    },
    {
        "name": "org_url",
        "type": "STRING",
        "description": "Org's profile URL"
    }
]

In [3]:
schema

[{'name': 'id', 'type': 'STRING', 'description': 'Unique event ID'},
 {'name': 'type',
  'type': 'STRING',
  'description': 'https://developer.github.com/v3/activity/events/types/'},
 {'name': 'actor_id',
  'type': 'INTEGER',
  'description': 'Numeric ID of the GitHub actor'},
 {'name': 'actor_login',
  'type': 'STRING',
  'description': "Actor's GitHub login"},
 {'name': 'actor_gravatar_id',
  'type': 'STRING',
  'description': "Actor's Gravatar ID"},
 {'name': 'actor_url', 'type': 'STRING', 'description': "Actor's profile URL"},
 {'name': 'actor_avatar_url',
  'type': 'STRING',
  'description': "Actor's Gravatar URL"},
 {'name': 'repo_id',
  'type': 'INTEGER',
  'description': 'Numeric ID of the GitHub repository'},
 {'name': 'repo_name', 'type': 'STRING', 'description': 'Repository name'},
 {'name': 'repo_url', 'type': 'STRING', 'description': 'Repository URL'},
 {'name': 'public',
  'type': 'BOOLEAN',
  'description': 'Always true for this dataset since only public activity is reco

In [4]:
spec_org = {
    'id':'id',
    'type':'type',
    'actor_id':'actor.id',
    'actor_login':'actor.login',
    'actor_gravatar_id':'actor.gravatar_id',
    'actor_url':'actor.url',
    'actor_avatar_url':'actor.avatar_url',
    'repo_id':'repo.id',
    'repo_name':'repo.name',
    'repo_url':'repo.url',
    #'payload':'payload',
    'public':'public',
    'created_at':'created_at',
    'org_id':'org.id',
    'org_login':'org.login',
    'org_gravatar_id':'org.gravatar_id',
    'org_avatar_url':'org.avatar_url',
    'org_url':'org.url'
}

In [5]:
spec = {
    'id':'id',
    'type':'type',
    'actor_id':'actor.id',
    'actor_login':'actor.login',
    'actor_gravatar_id':'actor.gravatar_id',
    'actor_url':'actor.url',
    'actor_avatar_url':'actor.avatar_url',
    'repo_id':'repo.id',
    'repo_name':'repo.name',
    'repo_url':'repo.url',
    #'payload':'payload',
    'public':'public',
    'created_at':'created_at'
}

In [37]:
df.iloc[5].org

{'id': 66639503,
 'login': 'sepinf-inc',
 'gravatar_id': '',
 'url': 'https://api.github.com/orgs/sepinf-inc',
 'avatar_url': 'https://avatars.githubusercontent.com/u/66639503?'}

In [38]:
df.iloc[5].actor

{'id': 7276994,
 'login': 'lfcnassif',
 'display_login': 'lfcnassif',
 'gravatar_id': '',
 'url': 'https://api.github.com/users/lfcnassif',
 'avatar_url': 'https://avatars.githubusercontent.com/u/7276994?'}

In [39]:
print(glom(df.iloc[5], spec))

{'id': 20926542675, 'type': 'IssueCommentEvent', 'actor_id': 7276994, 'actor_login': 'lfcnassif', 'actor_gravatar_id': '', 'actor_url': 'https://api.github.com/users/lfcnassif', 'actor_avatar_url': 'https://avatars.githubusercontent.com/u/7276994?', 'repo_id': 37318294, 'repo_name': 'sepinf-inc/IPED', 'repo_url': 'https://api.github.com/repos/sepinf-inc/IPED', 'public': True, 'created_at': Timestamp('2022-03-25 00:00:00+0000', tz='UTC')}


In [40]:
temp = glom(df.iloc[5], spec)

In [41]:
if df.iloc[5].org is np.nan:
    print('derp')
else:
    print('dorp')

dorp


In [27]:
data = []
index = ['id', 'type', 'actor_id', 'actor_login', 'actor_gravatar_id', 'actor_url', 'actor_avatar_url', 'repo_id', 'repo_name', 'repo_url', 'public', 'created_at', 'org_id', 'org_login', 'org_gravatar_id', 'org_avatar_url', 'org_url']
file='test3.json'
with open(file) as f:
    for line in f:
        j_content = json.loads(line)
        if j_content.get('org') is None:
            d_line = glom (j_content, spec)
        else:
            d_line = glom(j_content, spec_org)
        data.append(d_line)

In [ ]:
data = []
file='2022-03-30-23.json.gz'
with gzip.open(file, 'rt', encoding='UTF-8') as f:
    for line in f:
        j_content = json.loads(line)
        if j_content.get('org') is None:
            d_line = glom (j_content, spec)
        else:
            d_line = glom(j_content, spec_org)
        data.append(d_line)

In [28]:
dff = pd.DataFrame(data)

In [30]:
# convert blanks to NaN
dff = dff.mask(dff == '')

In [31]:
# convert to int
dff.org_id = dff.org_id.astype('int64', errors='ignore')

In [33]:
series = pd.notnull(dff.org_id)
dff[series]

,id,type,actor_id,actor_login,actor_gravatar_id,actor_url,actor_avatar_url,repo_id,repo_name,repo_url,public,created_at,org_id,org_login,org_gravatar_id,org_avatar_url,org_url
5,20926542675,IssueCommentEvent,7276994,lfcnassif,NaN,https://api.github.com/users/lfcnassif,https://avatars.githubusercontent.com/u/7276994?,37318294,sepinf-inc/IPED,https://api.github.com/repos/sepinf-inc/IPED,True,2022-03-25T00:00:00Z,66639503.0,sepinf-inc,NaN,https://avatars.githubusercontent.com/u/66639503?,https://api.github.com/orgs/sepinf-inc
8,20926542695,PushEvent,52905881,giga-a,NaN,https://api.github.com/users/giga-a,https://avatars.githubusercontent.com/u/52905881?,317319347,qeeqbox/social-analyzer,https://api.github.com/repos/qeeqbox/social-an...,True,2022-03-25T00:00:00Z,54420737.0,qeeqbox,NaN,https://avatars.githubusercontent.com/u/54420737?,https://api.github.com/orgs/qeeqbox
9,20926542700,PullRequestReviewEvent,33912936,errose28,NaN,https://api.github.com/users/errose28,https://avatars.githubusercontent.com/u/33912936?,212382406,apache/ozone,https://api.github.com/repos/apache/ozone,True,2022-03-25T00:00:00Z,47359.0,apache,NaN,https://avatars.githubusercontent.com/u/47359?,https://api.github.com/orgs/apache
10,20926542703,PullRequestReviewEvent,33912936,errose28,NaN,https://api.github.com/users/errose28,https://avatars.githubusercontent.com/u/33912936?,212382406,apache/ozone,https://api.github.com/repos/apache/ozone,True,2022-03-25T00:00:00Z,47359.0,apache,NaN,https://avatars.githubusercontent.com/u/47359?,https://api.github.com/orgs/apache
11,20926542708,IssueCommentEvent,1185309,xing-yang,NaN,https://api.github.com/users/xing-yang,https://avatars.githubusercontent.com/u/1185309?,351669747,kubernetes-csi/lib-volume-populator,https://api.github.com/repos/kubernetes-csi/li...,True,2022-03-25T00:00:00Z,33050221.0,kubernetes-csi,NaN,https://avatars.githubusercontent.com/u/33050221?,https://api.github.com/orgs/kubernetes-csi
12,20926542709,PushEvent,64379712,amandagonsalves,NaN,https://api.github.com/users/amandagonsalves,https://avatars.githubusercontent.com/u/64379712?,350340084,syscoin/pali-wallet,https://api.github.com/repos/syscoin/pali-wallet,True,2022-03-25T00:00:00Z,6983571.0,syscoin,NaN,https://avatars.githubusercontent.com/u/6983571?,https://api.github.com/orgs/syscoin
16,20926542747,ForkEvent,55965636,biologioholic,NaN,https://api.github.com/users/biologioholic,https://avatars.githubusercontent.com/u/55965636?,112323127,alan-turing-institute/mlaut,https://api.github.com/repos/alan-turing-insti...,True,2022-03-25T00:00:00Z,18304793.0,alan-turing-institute,NaN,https://avatars.githubusercontent.com/u/18304793?,https://api.github.com/orgs/alan-turing-institute
17,20926542752,PushEvent,63430363,JoaquimAlt,NaN,https://api.github.com/users/JoaquimAlt,https://avatars.githubusercontent.com/u/63430363?,472952533,cyber-rhino/CarrosTop-Project,https://api.github.com/repos/cyber-rhino/Carro...,True,2022-03-25T00:00:00Z,102097672.0,cyber-rhino,NaN,https://avatars.githubusercontent.com/u/102097...,https://api.github.com/orgs/cyber-rhino
21,20926542771,PullRequestEvent,709500,Porkepix,NaN,https://api.github.com/users/Porkepix,https://avatars.githubusercontent.com/u/709500?,52855516,Homebrew/homebrew-core,https://api.github.com/repos/Homebrew/homebrew...,True,2022-03-25T00:00:00Z,1503512.0,Homebrew,NaN,https://avatars.githubusercontent.com/u/1503512?,https://api.github.com/orgs/Homebrew
25,20926542808,PushEvent,44138938,FunixG,NaN,https://api.github.com/users/FunixG,https://avatars.githubusercontent.com/u/44138938?,229434607,FunixProductions/FunixBot,https://api.github.com/repos/FunixProductions/...,True,2022-03-25T00:00:01Z,99349214.0,FunixProductions,NaN,https://avatars.githubusercontent.com/u/99349214?,https://api.github.com/orgs/FunixProductions


In [7]:
dff.to_parquet('2022-03-30-23.parquet')

In [11]:
def json2parquet(src_file):
    if not src_file.endswith('.json.gz'):
        logging.error("Can only accept source files in json.gz format, for the moment")
        return
    data = []
    with gzip.open(src_file, 'rt', encoding='UTF-8') as f:
        for line in f:
            j_content = json.loads(line)
            if j_content.get('org') is None:
                d_line = glom (j_content, spec)
            else:
                d_line = glom(j_content, spec_org)
            data.append(d_line)
    pd.DataFrame(data).to_parquet(src_file.replace('.json.gz', '.parquet'))

In [12]:
json2parquet('2022-03-30-23.json.gz')

In [100]:
f = gzip.open('2022-03-30-23.json.gz', 'rt', encoding='UTF-8')

In [102]:
f.read()

In [98]:
df = pd.json_normalize(f)

In [92]:
index = ['id', 'type', 'actor.id', 'actor.login', 'actor.gravatar_id', 'actor.url', 'actor.avatar_url', 'repo.id', 'repo.name', 'repo.url', 'public', 'created_at', 'org.id', 'org.login', 'org.gravatar_id', 'org.avatar_url', 'org.url']
df = df.reindex(columns=index)

In [93]:
df

,id,type,actor.id,actor.login,actor.gravatar_id,actor.url,actor.avatar_url,repo.id,repo.name,repo.url,public,created_at,org.id,org.login,org.gravatar_id,org.avatar_url,org.url
0,21029121984,PullRequestEvent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2022-03-30 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN
1,21029121985,PushEvent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2022-03-30 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2,21029121989,CreateEvent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2022-03-30 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN
3,21029121990,PushEvent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2022-03-30 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN
4,21029121994,PullRequestReviewEvent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2022-03-30 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267568,21030056144,PullRequestReviewCommentEvent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2022-03-30 23:52:52+00:00,NaN,NaN,NaN,NaN,NaN
267569,21030056165,PullRequestReviewCommentEvent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2022-03-30 23:56:36+00:00,NaN,NaN,NaN,NaN,NaN
267570,21030056124,PullRequestReviewCommentEvent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2022-03-30 23:57:09+00:00,NaN,NaN,NaN,NaN,NaN
267571,21030056127,PullRequestReviewCommentEvent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2022-03-30 23:57:24+00:00,NaN,NaN,NaN,NaN,NaN


In [86]:
data = []
index = ['id', 'type', 'actor.id', 'actor.login', 'actor.gravatar_id', 'actor.url', 'actor.avatar_url', 'repo.id', 'repo.name', 'repo.url', 'public', 'created_at', 'org.id', 'org.login', 'org.gravatar_id', 'org.avatar_url', 'org.url']
with gzip.open('2022-03-30-23.json.gz', 'rt', encoding='UTF-8') as f:
    for line in f:
        j_line = json.loads(line)
        flat = json_normalize(j_line)
        flat = flat.reindex(columns=index)
        data.append(flat.to_dict('records')[0])


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(data)
df